In [2]:
import pandas as pd
import sqlite3

from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB

In [44]:
from sklearn.model_selection import train_test_split

In [3]:
conn = sqlite3.connect('data.sqlite3')

In [18]:
us_fac = pd.read_sql_query("""
SELECT uf.faculty_id, subs_id FROM users_subscriptions AS us
JOIN users_faculties as uf ON uf.user_id = us.user_id
WHERE uf.faculty_id in 
(SELECT DISTINCT faculty_id from faculties)
""", con=conn)

In [34]:
us_fac = us_fac[~us_fac.duplicated()]

In [35]:
us_fac

,faculty_id,subs_id
0,2470,1023421
1,2470,121768
2,2470,1248637
3,2470,127659593
4,2470,14305889
...,...,...
849395,2231285,62400590
849396,2231285,65525231
849397,2231285,67132434
849398,2231285,6925581


In [36]:
one = OneHotEncoder()

In [59]:
fitted = one.fit_transform(us_fac['subs_id'].to_numpy().reshape(-1, 1)).toarray()

In [72]:
fitted_df = pd.DataFrame(fitted)

In [116]:
facids = us_fac[['faculty_id']].reset_index()

In [136]:
prepared =  pd.concat([facids, fitted_df ], axis=1).drop(columns='index')

In [137]:
X, y = train_test_split(prepared)

In [138]:
bayes = GaussianNB()
#.loc[:, X.columns != 'faculty_id']
#.loc[:, X.columns != 'faculty_id']

In [142]:
bayes.fit(X, X['faculty_id'])

GaussianNB()

In [143]:
bayes.score(y, y['faculty_id'])

0.5764705882352941

In [144]:
prepared

,faculty_id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,2470,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2470,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2470,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2470,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2470,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,2231285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3396,2231285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3397,2231285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3398,2231285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [145]:
conn.close()